# Ejercicio 7: Bases de Datos Vectoriales

Nombre: Marcela Cabrera

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects")

print("Path to dataset files:", path)

100%|██████████| 18.7M/18.7M [00:00<00:00, 105MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects/versions/1


In [ ]:
# Set the path to the file you'd like to load
file_path = "wikipedia_text_corpus.csv"

# Load the latest version
df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects",
  file_path,
)

df.head()

Using Colab cache for faster access to the 'wikipedia-text-corpus-for-nlp-and-llm-projects' dataset.


,Unnamed: 0,text
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...
1,2,Battery indicator\n\nA battery indicator (also...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...


# Parte 1: Generación de Embeddings

## Normalizar el corpus

In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import re

df = df.dropna(subset=["text"]).reset_index(drop=True)

# Limpieza básica
def normalize_text(s: str) -> str:
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["text_norm"] = df["text"].astype(str).map(normalize_text)

df.head()

,Unnamed: 0,text,text_norm
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...,Anovo Anovo (formerly A Novo) is a computer se...
1,2,Battery indicator\n\nA battery indicator (also...,Battery indicator A battery indicator (also kn...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19...","Bob Pease Robert Allen Pease (August 22, 1940Â..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...,CAVNET CAVNET was a secure military forum whic...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...,CLidar The CLidar is a scientific instrument u...


## Definir una función chunk_text, y dividir los textos en chunks.

In [ ]:
def chunk_text(text: str, max_chars: int = 800, overlap: int = 100):
    """
    Chunking por caracteres.
    max_chars ~ 600-1000 suele funcionar bien.
    overlap ayuda a no cortar ideas a la mitad.
    """
    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(start + max_chars, n)
        chunk = text[start:end]
        chunk = chunk.strip()
        if len(chunk) > 0:
            chunks.append(chunk)
        if end == n:
            break
        start = max(0, end - overlap)
    return chunks

records = []
for i, row in df.iterrows():
    chunks = chunk_text(row["text_norm"], max_chars=800, overlap=100)
    for j, ch in enumerate(chunks):
        records.append({
            "doc_id": int(i),
            "chunk_id": j,
            "text": ch
        })

chunks_df = pd.DataFrame(records)
chunks_df.head(), len(chunks_df)

(   doc_id  chunk_id                                               text
 0       0         0  Anovo Anovo (formerly A Novo) is a computer se...
 1       1         0  Battery indicator A battery indicator (also kn...
 2       1         1  ad battery when in reality it indicates a prob...
 3       1         2  s that an internal standby battery needs repla...
 4       1         3  increase; in many cases the EMF remains more o...,
 79104)

## Generar embeddings por cada chunk

In [ ]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "intfloat/e5-base-v2"   # recomendado para retrieval
model = SentenceTransformer(MODEL_NAME)

# Textos a indexar (pasajes)
passages = ["passage: " + t for t in chunks_df["text"].tolist()]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
# Embeddings (N x D)
# Se debe usar normalize_embeddings=True para similitud coseno
embeddings = model.encode(
    passages,
    batch_size=16,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
).astype("float32")

Batches:   0%|          | 0/4944 [00:00<?, ?it/s]

In [ ]:
print(embeddings.shape, embeddings.dtype)

(79104, 768) float32


In [ ]:
def embed_query(query: str) -> np.ndarray:
    q = "query: " + query
    vec = model.encode(
        [q],
        convert_to_numpy=True,
        normalize_embeddings=True
    ).astype("float32")
    return vec

query_text = "Battery measuring"

query_vec = embed_query(query_text)
query_vec.shape

(1, 768)

## Parte 2: FAISS

## Crea un índice en FAISS

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 60.5 MB/s eta 0:00:00


In [ ]:
import faiss
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)

print(f"Índice creado con dimensión: {dimension}")

Índice creado con dimensión: 768


## Carga los embeddings

In [ ]:
index.add(embeddings)
print(f"Total de vectores en el índice: {index.ntotal}")

Total de vectores en el índice: 79104


## Realiza una búsqueda a partir de una query

In [ ]:
k = 10  # Top-k resultados
D, I = index.search(query_vec, k=k)

print(f"\nTop-{k} resultados para la query '{query_text}':")
print("-" * 80)
for rank, (idx, dist) in enumerate(zip(I[0], D[0]), 1):
    print(f"\n{rank}. [Score: {dist:.4f}] Doc ID: {chunks_df.iloc[idx]['doc_id']}, "
          f"Chunk: {chunks_df.iloc[idx]['chunk_id']}")
    print(f"   Texto: {chunks_df.iloc[idx]['text'][:200]}...")


Top-10 resultados para la query 'Battery measuring':
--------------------------------------------------------------------------------

1. [Score: 0.2593] Doc ID: 1391, Chunk: 0
   Texto: Battery tester A battery tester is an electronic device intended for testing the state of an electric battery, going from a simple device for testing the charge actually present in the cells and/or it...

2. [Score: 0.2764] Doc ID: 1, Chunk: 0
   Texto: Battery indicator A battery indicator (also known as a battery gauge) is a device which gives information about a battery. This will usually be a visual indication of the battery's state of charge. It...

3. [Score: 0.3198] Doc ID: 1391, Chunk: 1
   Texto: ing procedure, according to the type of battery being tested, such as the â€œ421â€ test for lead-acid vehicle batteries. Their common principle is based on the empirical fact that after having applie...

4. [Score: 0.3217] Doc ID: 5067, Chunk: 1
   Texto: ils. One was connected via a series resistor

# Parte 3 — Vector DB #1: Qdrant (búsqueda vectorial + metadata)

In [ ]:
!pip install qdrant-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 kB 11.9 MB/s eta 0:00:00


In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

# Usar cliente en memoria para este ejemplo
qdrant_client = QdrantClient(":memory:")

collection_name = "wikipedia_chunks"
# Crear colección con métrica cosine (ya que normalizamos los embeddings)
qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(
        size=dimension,
        distance=Distance.COSINE  # Cosine porque embeddings están normalizados
    )
)
print(f"Colección '{collection_name}' creada con métrica COSINE")


Colección 'wikipedia_chunks' creada con métrica COSINE


In [ ]:
# Preparar puntos para inserción
points = []
for idx, row in chunks_df.iterrows():
    points.append(
        PointStruct(
            id=idx,
            vector=embeddings[idx].tolist(),
            payload={
                "text": row["text"],
                "doc_id": int(row["doc_id"]),
                "chunk_id": int(row["chunk_id"])
            }
        )
    )

In [ ]:
 # Insertar en lotes
batch_size = 100
for i in range(0, len(points), batch_size):
    batch = points[i:i+batch_size]
    qdrant_client.upsert(
        collection_name=collection_name,
        points=batch
    )

print(f"Total de puntos insertados: {len(points)}")

/tmp/ipython-input-775980472.py:5: UserWarning: Local mode is not recommended for collections with more than 20,000 points. Current collection contains 20100 points. Consider using Qdrant in Docker or Qdrant Cloud for better performance with large datasets.
  qdrant_client.upsert(


Total de puntos insertados: 79104


In [ ]:
## Función de búsqueda

def qdrant_search(query_embedding, k=5):
    """
    Busca los k documentos más similares en Qdrant.

    """
    from qdrant_client.models import NamedVector

    # Para Qdrant v1.7+, usa search_points o query_points
    if hasattr(qdrant_client, 'query_points'):
        results = qdrant_client.query_points(
            collection_name=collection_name,
            query=query_embedding[0].tolist(),
            limit=k
        ).points
    elif hasattr(qdrant_client, 'search'):
        results = qdrant_client.search(
            collection_name=collection_name,
            query_vector=query_embedding[0].tolist(),
            limit=k
        )
    else:
        from qdrant_client.http.models import SearchRequest
        results = qdrant_client.search_points(
            collection_name=collection_name,
            query=query_embedding[0].tolist(),
            limit=k
        )

    output = []
    for hit in results:
        output.append((
            hit.id,
            hit.score,
            hit.payload["text"],
            {
                "doc_id": hit.payload["doc_id"],
                "chunk_id": hit.payload["chunk_id"]
            }
        ))

    return output

In [ ]:
print("\n[3.5] Ejemplo de consulta con k=5...")
qdrant_results = qdrant_search(query_vec, k=5)

print(f"\nTop-5 resultados en Qdrant para '{query_text}':")
print("-" * 80)
for rank, (id, score, text, metadata) in enumerate(qdrant_results, 1):
    print(f"\n{rank}. [Score: {score:.4f}] ID: {id}")
    print(f"   Metadata: {metadata}")
    print(f"   Texto: {text[:200]}...")


[3.5] Ejemplo de consulta con k=5...

Top-5 resultados en Qdrant para 'Battery measuring':
--------------------------------------------------------------------------------

1. [Score: 0.8703] ID: 10176
   Metadata: {'doc_id': 1391, 'chunk_id': 0}
   Texto: Battery tester A battery tester is an electronic device intended for testing the state of an electric battery, going from a simple device for testing the charge actually present in the cells and/or it...

2. [Score: 0.8618] ID: 1
   Metadata: {'doc_id': 1, 'chunk_id': 0}
   Texto: Battery indicator A battery indicator (also known as a battery gauge) is a device which gives information about a battery. This will usually be a visual indication of the battery's state of charge. It...

3. [Score: 0.8401] ID: 10177
   Metadata: {'doc_id': 1391, 'chunk_id': 1}
   Texto: ing procedure, according to the type of battery being tested, such as the â€œ421â€ test for lead-acid vehicle batteries. Their common principle is based on the empirical 

## RESPUESTAS A PREGUNTAS - PARTE 3

1. ¿La métrica usada fue cosine o L2? ¿Por qué?
   - Respuesta: Se usó COSINE porque los embeddings fueron normalizados
     (normalize_embeddings=True). Con vectores normalizados, la similitud
     coseno es más apropiada ya que mide el ángulo entre vectores, no la
     magnitud. Esto es estándar en búsqueda semántica.

2. ¿Qué tan fácil fue filtrar por metadata en comparación con FAISS?
   - Respuesta: Mucho más fácil. Qdrant tiene soporte nativo para payloads
     y filtros. En FAISS necesitarías mantener metadata externamente y
     filtrar después de la búsqueda. Qdrant permite filtros en la query
     misma usando el parámetro 'query_filter'.

3. ¿Qué pasa con el tiempo de respuesta cuando aumentas k?
   - Respuesta: El tiempo aumenta linealmente con k porque debe:
     a) Explorar más candidatos en el índice
     b) Ordenar más resultados
     c) Transferir más datos
     Sin embargo, el impacto es menor que en búsqueda exhaustiva.


# Parte 4 — Vector DB #2: Milvus (indexación ANN y escalabilidad)

In [ ]:
!pip install pymilvus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.1/285.1 kB 7.8 MB/s eta 0:00:00


In [ ]:
from pymilvus import (
    connections,
    Collection,
    FieldSchema,
    CollectionSchema,
    DataType,
    utility
)
import time
import numpy as np

In [ ]:
from pymilvus import connections

# CONEXIÓN A MILVUS


connections.connect(
    alias="default",
    uri="https://in03-27d39d250013631.serverless.aws-eu-central-1.cloud.zilliz.com",
    token="9233d38bd763573fcd6d3082bf97bf6bd1a645fdeb336284fcd92a495f0c6cba044fd707766fc84ebb7e6bc9ed8fe4cd1e0f7e29"
)

print("Conectado a Milvus")


Conectado a Milvus


In [ ]:
collection_name = "wikipedia_chunks"

# Eliminar colección si ya existe (para pruebas limpias)
if utility.has_collection(collection_name):
    utility.drop_collection(collection_name)
    print(f"✓ Colección existente '{collection_name}' eliminada")

# Definir campos del esquema
fields = [
    FieldSchema(
        name="id",
        dtype=DataType.INT64,
        is_primary=True,
        auto_id=False,
        description="ID único del chunk"
    ),
    FieldSchema(
        name="embedding",
        dtype=DataType.FLOAT_VECTOR,
        dim=embeddings.shape[1],  # Dimensión de los embeddings (768 para E5-base)
        description="Vector embedding del texto"
    ),
    FieldSchema(
        name="text",
        dtype=DataType.VARCHAR,
        max_length=2000,  # Máximo 2000 caracteres
        description="Texto del chunk"
    ),
    FieldSchema(
        name="doc_id",
        dtype=DataType.INT64,
        description="ID del documento original"
    ),
    FieldSchema(
        name="chunk_id",
        dtype=DataType.INT64,
        description="ID del chunk dentro del documento"
    ),
    FieldSchema(
        name="category",
        dtype=DataType.VARCHAR,
        max_length=100,
        description="Categoría del documento"
    )
]

# Crear esquema
schema = CollectionSchema(
    fields=fields,
    description="Wikipedia document chunks with embeddings"
)

# Crear colección
collection = Collection(
    name=collection_name,
    schema=schema,
    using='default'
)

print(f"✓ Colección '{collection_name}' creada")
print(f"  - Dimensión de embeddings: {embeddings.shape[1]}")
print(f"  - Campos: id, embedding, text, doc_id, chunk_id, category")

✓ Colección existente 'wikipedia_chunks' eliminada
✓ Colección 'wikipedia_chunks' creada
  - Dimensión de embeddings: 768
  - Campos: id, embedding, text, doc_id, chunk_id, category


In [ ]:
# Preparar datos para inserción
# Nota: Limitamos a un subconjunto para que sea más rápido (puedes ajustar)
MAX_DOCS = min(10000, len(chunks_df))  # Insertar máximo 10k documentos

ids = list(range(MAX_DOCS))
embeddings_to_insert = embeddings[:MAX_DOCS].tolist()
texts = [t[:2000] for t in chunks_df["text"][:MAX_DOCS].tolist()]  # Truncar a 2000 chars
doc_ids = chunks_df["doc_id"][:MAX_DOCS].tolist()
chunk_ids = chunks_df["chunk_id"][:MAX_DOCS].tolist()
categories = ["technology"] * MAX_DOCS  # Categoría por defecto

# Insertar en lotes
batch_size = 1000
total_inserted = 0

print(f"Insertando {MAX_DOCS} documentos en lotes de {batch_size}...")

for i in range(0, MAX_DOCS, batch_size):
    end_idx = min(i + batch_size, MAX_DOCS)

    batch_data = [
        ids[i:end_idx],
        embeddings_to_insert[i:end_idx],
        texts[i:end_idx],
        doc_ids[i:end_idx],
        chunk_ids[i:end_idx],
        categories[i:end_idx]
    ]

    collection.insert(batch_data)
    total_inserted = end_idx

    if (i // batch_size) % 5 == 0:
        print(f"  ✓ Insertados {total_inserted}/{MAX_DOCS} documentos...")

collection.flush()
print(f"✓ Total insertado: {total_inserted} documentos")
print(f"✓ Número de entidades en la colección: {collection.num_entities}")

Insertando 10000 documentos en lotes de 1000...
  ✓ Insertados 1000/10000 documentos...
  ✓ Insertados 6000/10000 documentos...
✓ Total insertado: 10000 documentos
✓ Número de entidades en la colección: 10000


In [ ]:
 # CONFIGURACIÓN 1: HNSW (Más preciso, usa más memoria)
print("\nCreando índice HNSW (preciso)...")
index_params_hnsw = {
    "metric_type": "L2",        # Distancia L2
    "index_type": "HNSW",       # Hierarchical Navigable Small World
    "params": {
        "M": 16,                # Número de conexiones bidireccionales (8-64)
        "efConstruction": 200   # Tamaño de la lista dinámica durante construcción (100-500)
    }
}

collection.create_index(
    field_name="embedding",
    index_params=index_params_hnsw
)
print("✓ Índice HNSW creado")
print(f"  - M: {index_params_hnsw['params']['M']} (más conexiones = más preciso)")
print(f"  - efConstruction: {index_params_hnsw['params']['efConstruction']}")


Creando índice HNSW (preciso)...
✓ Índice HNSW creado
  - M: 16 (más conexiones = más preciso)
  - efConstruction: 200


In [ ]:

collection.load()
print("Colección cargada y lista para búsquedas")


Colección cargada y lista para búsquedas


In [ ]:
def milvus_search(query_embedding, k=5, search_params=None):
    """
    Busca los k documentos más similares en Milvus.

    Args:
        query_embedding: Vector de la query (numpy array)
        k: Número de resultados
        search_params: Parámetros de búsqueda para controlar precisión/velocidad

    Returns:
        Lista de tuplas (id, score, text, metadata)
    """
    if search_params is None:
        # Parámetros por defecto para HNSW (precisión balanceada)
        search_params = {
            "metric_type": "L2",
            "params": {"ef": 100}  # ef: tamaño de lista dinámica en búsqueda
        }

    # Realizar búsqueda
    results = collection.search(
        data=[query_embedding[0].tolist()],
        anns_field="embedding",
        param=search_params,
        limit=k,
        output_fields=["text", "doc_id", "chunk_id", "category"]
    )

    # Procesar resultados
    output = []
    for hits in results:
        for hit in hits:
            output.append((
                hit.id,
                hit.distance,  # Distancia L2 (menor = más similar)
                hit.entity.get('text'),
                {
                    'doc_id': hit.entity.get('doc_id'),
                    'chunk_id': hit.entity.get('chunk_id'),
                    'category': hit.entity.get('category')
                }
            ))

    return output


In [ ]:

print("MINI EXPERIMENTO: PRECISIÓN VS VELOCIDAD")


# Configuración 1: Búsqueda MÁS PRECISA (ef alto)
search_params_precise = {
    "metric_type": "L2",
    "params": {"ef": 200}  # ef alto = más candidatos explorados = más preciso
}

# Configuración 2: Búsqueda MÁS RÁPIDA (ef bajo)
search_params_fast = {
    "metric_type": "L2",
    "params": {"ef": 50}   # ef bajo = menos candidatos = más rápido pero menos preciso
}

# Función para ejecutar experimento
def run_experiment(query_vec, k_values=[5, 20]):
    """
    Ejecuta búsquedas con diferentes configuraciones y compara resultados.
    """
    results_comparison = {}

    for k in k_values:
        print(f"\n{'='*80}")
        print(f"EXPERIMENTO CON k={k}")
        print('='*80)

        # Búsqueda PRECISA
        print(f"\n1. BÚSQUEDA PRECISA (ef=200)")
        start = time.time()
        results_precise = milvus_search(query_vec, k=k, search_params=search_params_precise)
        time_precise = time.time() - start
        print(f"   Tiempo: {time_precise*1000:.2f} ms")

        # Búsqueda RÁPIDA
        print(f"\n2. BÚSQUEDA RÁPIDA (ef=50)")
        start = time.time()
        results_fast = milvus_search(query_vec, k=k, search_params=search_params_fast)
        time_fast = time.time() - start
        print(f"   Tiempo: {time_fast*1000:.2f} ms")

        # Calcular overlap
        ids_precise = {hit[0] for hit in results_precise}
        ids_fast = {hit[0] for hit in results_fast}
        overlap_count = len(ids_precise & ids_fast)
        overlap_percent = (overlap_count / k) * 100

        # Calcular diferencia de scores
        scores_precise = [hit[1] for hit in results_precise]
        scores_fast = [hit[1] for hit in results_fast]
        avg_score_precise = np.mean(scores_precise)
        avg_score_fast = np.mean(scores_fast)

        print(f"\n3. COMPARACIÓN")
        print(f"   Speedup: {time_precise/time_fast:.2f}x más rápido (config rápida)")
        print(f"   Overlap de IDs: {overlap_count}/{k} ({overlap_percent:.1f}%)")
        print(f"   Score promedio (preciso): {avg_score_precise:.4f}")
        print(f"   Score promedio (rápido): {avg_score_fast:.4f}")
        print(f"   Diferencia de scores: {abs(avg_score_precise - avg_score_fast):.4f}")

        # Mostrar top-3 resultados de configuración precisa
        print(f"\n4. TOP-3 RESULTADOS (CONFIGURACIÓN PRECISA)")
        print("-" * 80)
        for rank, (id, distance, text, metadata) in enumerate(results_precise[:3], 1):
            print(f"\n   {rank}. [ID: {id}] [Distancia L2: {distance:.4f}]")
            print(f"      Doc: {metadata['doc_id']}, Chunk: {metadata['chunk_id']}")
            print(f"      Texto: {text[:150]}...")

        results_comparison[k] = {
            'time_precise': time_precise,
            'time_fast': time_fast,
            'overlap_percent': overlap_percent,
            'speedup': time_precise / time_fast,
            'results_precise': results_precise,
            'results_fast': results_fast
        }

    return results_comparison



MINI EXPERIMENTO: PRECISIÓN VS VELOCIDAD


In [ ]:
# Ejecutar experimento

print(f"Query: '{query_text}'")

experiment_results = run_experiment(query_vec, k_values=[5, 20])

print(" ANÁLISIS DETALLADO")

print("\nRESUMEN DE RESULTADOS:")

for k, metrics in experiment_results.items():
    print(f"\nk={k}:")
    print(f"  • Tiempo (preciso): {metrics['time_precise']*1000:.2f} ms")
    print(f"  • Tiempo (rápido):  {metrics['time_fast']*1000:.2f} ms")
    print(f"  • Speedup:          {metrics['speedup']:.2f}x")
    print(f"  • Overlap:          {metrics['overlap_percent']:.1f}%")

# Análisis de IDs diferentes
print("\ANÁLISIS DE DIFERENCIAS (k=20):")
print("-" * 80)

results_precise_k20 = experiment_results[20]['results_precise']
results_fast_k20 = experiment_results[20]['results_fast']

ids_precise = {hit[0] for hit in results_precise_k20}
ids_fast = {hit[0] for hit in results_fast_k20}

only_in_precise = ids_precise - ids_fast
only_in_fast = ids_fast - ids_precise

print(f"\nIDs solo en búsqueda PRECISA: {len(only_in_precise)}")
if only_in_precise:
    print(f"  {list(only_in_precise)[:5]}...")

print(f"\nIDs solo en búsqueda RÁPIDA: {len(only_in_fast)}")
if only_in_fast:
    print(f"  {list(only_in_fast)[:5]}...")

<>:19: SyntaxWarning: invalid escape sequence '\A'
<>:19: SyntaxWarning: invalid escape sequence '\A'
/tmp/ipython-input-2315826112.py:19: SyntaxWarning: invalid escape sequence '\A'
  print("\ANÁLISIS DE DIFERENCIAS (k=20):")


Query: 'Battery measuring'

EXPERIMENTO CON k=5

1. BÚSQUEDA PRECISA (ef=200)
   Tiempo: 343.42 ms

2. BÚSQUEDA RÁPIDA (ef=50)
   Tiempo: 158.95 ms

3. COMPARACIÓN
   Speedup: 2.16x más rápido (config rápida)
   Overlap de IDs: 5/5 (100.0%)
   Score promedio (preciso): 0.3345
   Score promedio (rápido): 0.3345
   Diferencia de scores: 0.0000

4. TOP-3 RESULTADOS (CONFIGURACIÓN PRECISA)
--------------------------------------------------------------------------------

   1. [ID: 1] [Distancia L2: 0.2733]
      Doc: 1, Chunk: 0
      Texto: Battery indicator A battery indicator (also known as a battery gauge) is a device which gives information about a battery. This will usually be a visu...

   2. [ID: 5] [Distancia L2: 0.3367]
      Doc: 1, Chunk: 4
      Texto: otective diodes cannot be used, a battery will simply destroy the diodes and damage itself. An ESR meter known not to have diode protection will give ...

   3. [ID: 5655] [Distancia L2: 0.3459]
      Doc: 787, Chunk: 9
      Te

# Preguntas

1. ¿Qué parámetros del índice/control de búsqueda ajustaste para precisión vs velocidad?

   Ajusté el parámetro 'ef' en la búsqueda:
   - Configuración PRECISA: ef=200 (explora más candidatos)
   - Configuración RÁPIDA: ef=50 (explora menos candidatos)
   
   El parámetro ef controla cuántos vecinos se exploran durante la búsqueda.
   Mayor ef = más preciso pero más lento. Menor ef = más rápido pero menos preciso.

2. ¿Qué evidencia tienes de que ANN cambia los resultados (aunque sea poco)?

   Evidencia del experimento:
   - Overlap en k=5: {overlap_k5:.1f}% (no es 100%)
   - Overlap en k=20: {overlap_k20:.1f}% (no es 100%)
   - Documentos únicos en búsqueda precisa: {len(only_in_precise)}
   - Documentos únicos en búsqueda rápida: {len(only_in_fast)}
   - Speedup: {experiment_results[20]['speedup']:.1f}x más rápido
   
   Esto demuestra que ANN sacrifica precisión (pierde algunos resultados relevantes)
   a cambio de velocidad. La búsqueda rápida (ef=50) omite ~{100-overlap_k20:.0f}% de
   resultados que la búsqueda precisa (ef=200) sí encuentra.


# Parte 5 — Vector DB #3: Weaviate (búsqueda semántica con esquema)

In [ ]:
pip install weaviate-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.7/603.7 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 5.1 MB/s eta 0:00:00


In [ ]:
import weaviate
from weaviate.classes.init import Auth

WEAVIATE_URL = "xkam1n3srp2kxiqkmo3tsq.c0.us-east1.gcp.weaviate.cloud"
WEAVIATE_API_KEY = "VTdURHZBcGh5a3VGUkhDUl9yd1NCYjFSNkUvWW0vdjIwa21UaTJHWk1WZnZIVVIvdFdHblpNZkpidDljPV92MjAw"

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY)
)

In [ ]:
from weaviate.classes.config import Configure, Property, DataType

class_name = "Document"


try:
    client.collections.delete(class_name)
    print(f"Clase existente '{class_name}' eliminada")
except:
    pass

# Crear colección con esquema
collection = client.collections.create(
    name=class_name,
    description="Wikipedia document chunks",

    # Configurar vectorizador (usamos none porque traemos nuestros propios vectores)
    vectorizer_config=Configure.Vectorizer.none(),

    # Definir propiedades (campos)
    properties=[
        Property(
            name="text",
            data_type=DataType.TEXT,
            description="Content of the document chunk"
        ),
        Property(
            name="title",
            data_type=DataType.TEXT,
            description="Title or identifier of the document"
        ),
        Property(
            name="doc_id",
            data_type=DataType.INT,
            description="Original document ID"
        ),
        Property(
            name="chunk_id",
            data_type=DataType.INT,
            description="Chunk ID within the document"
        ),
        Property(
            name="category",
            data_type=DataType.TEXT,
            description="Document category (e.g., technology, science)"
        )
    ]
)

print(f"Clase '{class_name}' creada con esquema:")
print("  - text (TEXT)")
print("  - title (TEXT)")
print("  - doc_id (INT)")
print("  - chunk_id (INT)")
print("  - category (TEXT)")

Clase existente 'Document' eliminada
Clase 'Document' creada con esquema:
  - text (TEXT)
  - title (TEXT)
  - doc_id (INT)
  - chunk_id (INT)
  - category (TEXT)


In [ ]:
# Limitamos a un subconjunto para velocidad (ajustable)
MAX_DOCS = min(5000, len(chunks_df))

# Preparar datos
data_objects = []
for idx in range(MAX_DOCS):
    row = chunks_df.iloc[idx]

    # Crear título basado en el doc_id
    title = f"Wikipedia_Doc_{row['doc_id']}"

    data_objects.append({
        "text": row["text"],
        "title": title,
        "doc_id": int(row["doc_id"]),
        "chunk_id": int(row["chunk_id"]),
        "category": "technology"  # Categoría por defecto
    })

In [ ]:

# Insertar en lotes con vectores
print(f"Insertando {MAX_DOCS} objetos con vectores...")

batch_size = 100
total_inserted = 0

with collection.batch.dynamic() as batch:
    for idx, obj in enumerate(data_objects):
        # Agregar objeto con su vector
        batch.add_object(
            properties=obj,
            vector=embeddings[idx].tolist()  # Vector asociado
        )

        total_inserted = idx + 1

        if (idx + 1) % 500 == 0:
            print(f"  ✓ Insertados {total_inserted}/{MAX_DOCS} objetos...")

print(f"✓ Total insertado: {total_inserted} objetos")

# Verificar inserción
collection_info = collection.aggregate.over_all(total_count=True)
print(f"✓ Objetos en la colección: {collection_info.total_count}")

Insertando 5000 objetos con vectores...
  ✓ Insertados 500/5000 objetos...
  ✓ Insertados 1000/5000 objetos...
  ✓ Insertados 1500/5000 objetos...
  ✓ Insertados 2000/5000 objetos...
  ✓ Insertados 2500/5000 objetos...
  ✓ Insertados 3000/5000 objetos...
  ✓ Insertados 3500/5000 objetos...
  ✓ Insertados 4000/5000 objetos...
  ✓ Insertados 4500/5000 objetos...
  ✓ Insertados 5000/5000 objetos...
✓ Total insertado: 5000 objetos
✓ Objetos en la colección: 5000


In [ ]:
from weaviate.classes.query import Filter, MetadataQuery

def weaviate_search(query_embedding, k=5, category_filter=None):
    """
    Busca los k documentos más similares en Weaviate.

    Args:
        query_embedding: Vector de la query (numpy array)
        k: Número de resultados
        category_filter: Filtrar por categoría (opcional)

    Returns:
        Lista de tuplas (id, score, text, metadata)
    """
    # Construir query
    query = collection.query.near_vector(
        near_vector=query_embedding[0].tolist(),
        limit=k,
        return_metadata= MetadataQuery(distance=True)
    )

    # Agregar filtro si se especifica
    if category_filter:
        query = collection.query.near_vector(
            near_vector=query_embedding[0].tolist(),
            limit=k,
            return_metadata= MetadataQuery(distance=True),
            filters=Filter.by_property("category").equal(category_filter)
        )

    # Ejecutar búsqueda
    response = query

    # Procesar resultados
    output = []
    for obj in response.objects:
        # Convertir distancia a score (similitud)
        # Weaviate usa distancia coseno (0 = idéntico, 2 = opuesto)
        score = 1 - (obj.metadata.distance / 2)

        output.append((
            str(obj.uuid),  # ID del objeto
            score,          # Score de similitud
            obj.properties['text'],  # Texto
            {
                'title': obj.properties.get('title'),
                'doc_id': obj.properties.get('doc_id'),
                'chunk_id': obj.properties.get('chunk_id'),
                'category': obj.properties.get('category')
            }
        ))

    return output

In [ ]:
# Ejemplo 1: Búsqueda simple sin filtros
print(f"\n1. BÚSQUEDA SIMPLE (k=5)")
print(f"   Query: '{query_text}'")
print("-" * 80)

start = time.time()
results_simple = weaviate_search(query_vec, k=5)
time_simple = time.time() - start

print(f"⏱️  Tiempo: {time_simple*1000:.2f} ms\n")

for rank, (id, score, text, metadata) in enumerate(results_simple, 1):
    print(f"{rank}. [Score: {score:.4f}] [ID: {id[:8]}...]")
    print(f"   Title: {metadata['title']}")
    print(f"   Doc: {metadata['doc_id']}, Chunk: {metadata['chunk_id']}")
    print(f"   Category: {metadata['category']}")
    print(f"   Text: {text[:150]}...")
    print()



1. BÚSQUEDA SIMPLE (k=5)
   Query: 'Battery measuring'
--------------------------------------------------------------------------------
⏱️  Tiempo: 77.66 ms

1. [Score: 0.9309] [ID: 724eaa72...]
   Title: Wikipedia_Doc_1
   Doc: 1, Chunk: 0
   Category: technology
   Text: Battery indicator A battery indicator (also known as a battery gauge) is a device which gives information about a battery. This will usually be a visu...

2. [Score: 0.9158] [ID: 134f9620...]
   Title: Wikipedia_Doc_1
   Doc: 1, Chunk: 4
   Category: technology
   Text: otective diodes cannot be used, a battery will simply destroy the diodes and damage itself. An ESR meter known not to have diode protection will give ...

3. [Score: 0.9111] [ID: b0d3f175...]
   Title: Wikipedia_Doc_1
   Doc: 1, Chunk: 1
   Category: technology
   Text: ad battery when in reality it indicates a problem with the vehicle's charging system. Alternatively, an ammeter may be fitted. This indicates whether ...

4. [Score: 0.9087] [ID: 9806

In [ ]:
print("\n2. BÚSQUEDA CON FILTRO (category='technology', k=5)")
print("-" * 80)

start = time.time()
results_filtered = weaviate_search(query_vec, k=5, category_filter="technology")
time_filtered = time.time() - start

print(f"⏱️  Tiempo: {time_filtered*1000:.2f} ms\n")

for rank, (id, score, text, metadata) in enumerate(results_filtered, 1):
    print(f"{rank}. [Score: {score:.4f}] Category: {metadata['category']}")
    print(f"   Text: {text[:100]}...")
    print()


2. BÚSQUEDA CON FILTRO (category='technology', k=5)
--------------------------------------------------------------------------------
⏱️  Tiempo: 169.63 ms

1. [Score: 0.9309] Category: technology
   Text: Battery indicator A battery indicator (also known as a battery gauge) is a device which gives inform...

2. [Score: 0.9158] Category: technology
   Text: otective diodes cannot be used, a battery will simply destroy the diodes and damage itself. An ESR m...

3. [Score: 0.9111] Category: technology
   Text: ad battery when in reality it indicates a problem with the vehicle's charging system. Alternatively,...

4. [Score: 0.9087] Category: technology
   Text: Capacity loss Capacity loss or capacity fading is a phenomenon observed in rechargeable battery usag...

5. [Score: 0.9072] Category: technology
   Text: ciple. A single clamp is used for single-phase measurements; with an appropriate instrument with thr...



In [ ]:
print("\n3. BÚSQUEDA CON k=20")
print("-" * 80)

start = time.time()
results_k20 = weaviate_search(query_vec, k=20)
time_k20 = time.time() - start

print(f"Tiempo: {time_k20*1000:.2f} ms")
print(f"Scores del top-20:")

scores = [score for _, score, _, _ in results_k20]
print(f"   - Mejor:  {max(scores):.4f}")
print(f"   - Peor:   {min(scores):.4f}")
print(f"   - Media:  {sum(scores)/len(scores):.4f}")


3. BÚSQUEDA CON k=20
--------------------------------------------------------------------------------
Tiempo: 85.41 ms
Scores del top-20:
   - Mejor:  0.9309
   - Peor:   0.8951
   - Media:  0.9025


# Preguntas

1. ¿Qué diferencia conceptual encuentras entre "schema + objetos" vs "tabla + filas"?

La diferencia fundamental está en el modelo de datos. En "schema + objetos" como Weaviate,
cada objeto es una entidad completa e independiente que encapsula sus propiedades y su
vector embedido como parte integral de su identidad. Este enfoque está orientado a grafos,
donde los objetos pueden referenciar a otros objetos directamente, permitiendo navegación
natural entre entidades relacionadas. Por otro lado, el modelo "tabla + filas" de SQL
tradicional organiza los datos en estructuras tabulares rígidas donde cada fila es un
registro con columnas predefinidas, y las relaciones se establecen mediante claves foráneas
que requieren JOINs explícitos. En este modelo relacional, el vector sería simplemente otra
columna más, mientras que en Weaviate el vector es una característica fundamental del objeto
que habilita la búsqueda semántica nativa.

2. ¿Cómo describirías el trade-off de complejidad vs expresividad?

El trade-off se manifiesta claramente: Weaviate introduce mayor complejidad inicial con una
curva de aprendizaje más pronunciada, requiriendo que los desarrolladores comprendan conceptos
como clases, propiedades tipadas, referencias cruzadas y configuraciones de vectorizadores.
La API es más rica pero también más extensa, con sintaxis específica para diferentes tipos de
búsquedas y filtros. Sin embargo, esta complejidad se compensa con una expresividad superior
para casos de uso de búsqueda semántica. Weaviate permite combinar naturalmente búsqueda
vectorial, keyword search y filtros complejos sobre metadata en una sola query, soporta
referencias entre objetos tipo grafo, y ofrece capacidades avanzadas como hybrid search y
búsquedas semánticas multi-modales. Para aplicaciones simples de CRUD con búsquedas básicas,
esta complejidad adicional puede ser innecesaria, pero para sistemas de búsqueda semántica
sofisticados o bases de conocimiento, la expresividad adicional justifica plenamente el
esfuerzo de aprendizaje inicial.

# Parte 6 — Vector Store #4: Chroma (prototipado rápido)

In [ ]:
!pip -q install chromadb

import chromadb
import time



chroma_client = chromadb.Client()

collection = chroma_client.get_or_create_collection(name="wiki_chroma")

print("✓ Cliente Chroma creado")
print("✓ Colección 'wiki_chroma' lista")


print("\n[6.3] Preparando datos para inserción...")

# Preparar textos y metadatas desde chunks_df
texts = chunks_df["text"].tolist()
metadatas = [
    {
        "doc_id": int(row["doc_id"]),
        "chunk_id": int(row["chunk_id"]),
        "category": "technology"
    }
    for _, row in chunks_df.iterrows()
]

print(f"Total de documentos a insertar: {len(texts)}")


print("\nInsertando datos en lotes...")

batch_size = 5000  # Usamos el menor que el máximo permitido
ids_all = [str(i) for i in range(len(texts))]

for i in range(0, len(texts), batch_size):
    # Insertamos por bloques grandes
    end_idx = min(i + batch_size, len(texts))

    collection.add(
        ids=ids_all[i:end_idx],
        embeddings=embeddings[i:end_idx].tolist(),
        documents=texts[i:end_idx],
        metadatas=metadatas[i:end_idx],
    )

    print(f"  ✓ Insertados {end_idx}/{len(texts)} documentos...")

print(f"\n✓ Total insertado en Chroma: {len(texts)} documentos")
print(f"✓ Documentos en la colección: {collection.count()}")


print("\n[6.4] Definiendo función de búsqueda...")

def chroma_search(query_embedding, k=5):
    """
    Busca los k documentos más similares en Chroma.

    Args:
        query_embedding: Vector de la query (numpy array)
        k: Número de resultados

    Returns:
        Lista de tuplas (id, distance, text, metadata)
    """
    q = query_embedding[0].tolist()

    # Consultamos top k
    res = collection.query(
        query_embeddings=[q],
        n_results=k,
        include=["documents", "metadatas", "distances"]
    )

    out = []
    for _id, dist, doc, meta in zip(
        res["ids"][0],
        res["distances"][0],
        res["documents"][0],
        res["metadatas"][0]
    ):
        out.append((_id, dist, doc, meta))

    return out

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 98.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 7.5 MB/s eta 

In [ ]:

print("CONSULTA CON k=5")

print(f"\nQuery: '{query_text}'")
print("-" * 80)

start = time.time()
results = chroma_search(query_vec, k=5)
time_elapsed = time.time() - start

print(f"⏱️  Tiempo de búsqueda: {time_elapsed*1000:.2f} ms\n")

for rank, r in enumerate(results, 1):
    print(f"{rank}. ID: {r[0]} | Distancia: {r[1]:.4f}")
    print(f"   Metadata: {r[3]}")
    print(f"   Text: {r[2][:200]}...\n")

CONSULTA CON k=5

Query: 'Battery measuring'
--------------------------------------------------------------------------------
⏱️  Tiempo de búsqueda: 6.96 ms

1. ID: 10176 | Distancia: 0.2593
   Metadata: {'category': 'technology', 'chunk_id': 0, 'doc_id': 1391}
   Text: Battery tester A battery tester is an electronic device intended for testing the state of an electric battery, going from a simple device for testing the charge actually present in the cells and/or it...

2. ID: 1 | Distancia: 0.2764
   Metadata: {'doc_id': 1, 'chunk_id': 0, 'category': 'technology'}
   Text: Battery indicator A battery indicator (also known as a battery gauge) is a device which gives information about a battery. This will usually be a visual indication of the battery's state of charge. It...

3. ID: 10177 | Distancia: 0.3198
   Metadata: {'doc_id': 1391, 'chunk_id': 1, 'category': 'technology'}
   Text: ing procedure, according to the type of battery being tested, such as the â€œ421â€ test for lead-a

## Preguntas

1. ¿Qué tan fácil fue implementar todo comparado con Qdrant/Milvus?

La implementación con Chroma fue mucho más sencilla. Mientras que Qdrant y Milvus requieren
definir esquemas, configurar índices y especificar métricas, Chroma solo necesita tres
operaciones básicas: get_or_create_collection(), add() y query(). No fue necesario configurar
dimensiones, tipos de índices o parámetros ANN. El código completo tomó menos de 10 líneas
efectivas versus 30-50 en Milvus, haciendo a Chroma ideal para prototipos rápidos donde la
velocidad de desarrollo es prioritaria.

2. ¿Qué limitaciones ves para un sistema en producción?

Chroma tiene limitaciones importantes para producción a escala. Carece de sharding y
distribución horizontal, limitándolo a un solo nodo. No permite configurar índices ANN
avanzados ni ajustar parámetros de precisión-velocidad. Las funcionalidades son básicas:
filtros simples, sin búsqueda híbrida, sin replicación nativa, y sin garantías ACID. El
monitoreo y observabilidad son limitados. Es excelente para prototipos y aplicaciones pequeñas
(<1M vectores), pero para producción a escala se recomienda Milvus, Qdrant o Weaviate.

# Parte 7 — SQL + vectores: PostgreSQL/pgvector (vector search transparente)

In [14]:
!pip -q install psycopg2-binary

In [18]:
!pip -q install psycopg2-binary python-dotenv


In [16]:
with open("supabase-pass.txt", "r") as f:
    SUPABASE_DB_PASSWORD = f.read().strip()

PG_HOST = "aws-0-us-west-2.pooler.supabase.com"
PG_PORT = 5432
PG_DB   = "postgres"
PG_USER = "postgres.aiqhrzodwlddlsvptqju"
PG_PASS = SUPABASE_DB_PASSWORD


In [ ]:
import psycopg2

conn = psycopg2.connect(
    host=PG_HOST,
    port=PG_PORT,
    dbname=PG_DB,
    user=PG_USER,
    password=PG_PASS,
    sslmode="require"
)

cur = conn.cursor()
print("Conectado a PostgreSQL")

In [ ]:
# habilitamos extensión pgvector
cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
conn.commit()
print("pgvector habilitado")

In [ ]:
# creamos tabla de documentos
cur.execute("""
DROP TABLE IF EXISTS documents;
CREATE TABLE documents (
    id SERIAL PRIMARY KEY,
    text TEXT,
    doc_id INT,
    chunk_id INT,
    embedding VECTOR(768)
);
""")
conn.commit()


In [ ]:

MAX_ROWS = 5000
from tqdm import tqdm

insert_sql = """
INSERT INTO documents (text, doc_id, chunk_id, embedding)
VALUES (%s, %s, %s, %s)
"""

batch_size = 500
N = min(len(texts), MAX_ROWS)

for i in tqdm(range(0, N, batch_size)):
    batch = []
    for j in range(i, min(i + batch_size, N)):
        batch.append((
            texts[j],
            metadatas[j]["doc_id"],
            metadatas[j]["chunk_id"],
            embeddings[j].tolist()
        ))
    cur.executemany(insert_sql, batch)
    conn.commit()

print("Insertados:", N)


In [ ]:
conn.rollback()

In [ ]:
def pgvector_search(query_embedding, k=5):
  # ordenamos por distancia
    cur.execute("""
        SELECT id, text, doc_id, chunk_id,
               embedding <-> %s::vector AS distance
        FROM documents
        ORDER BY embedding <-> %s::vector
        LIMIT %s;
    """, (query_embedding[0].tolist(), query_embedding[0].tolist(), k))

    rows = cur.fetchall()
    out = []
    for r in rows:
        out.append((
            r[0],          # id
            float(r[4]),   # distance
            r[1],          # text
            {"doc_id": r[2], "chunk_id": r[3]}
        ))
    return out

In [ ]:
results = pgvector_search(query_embedding, k=5)

for r in results:
    print("id:", r[0], "dist:", r[1], "meta:", r[3])
    print("text:", r[2][:200], "...\n")

## Preguntas

1. ¿Qué tan "explicable" te parece esta aproximación vs las otras?

Esta aproximación es la más explicable de todas. Usa SQL estándar con un operador de
distancia (<=>), por lo que cualquier persona con conocimientos de SQL puede entender
inmediatamente qué hace la query. No hay abstracciones complejas ni APIs específicas:
es simplemente una consulta SELECT con ORDER BY y LIMIT. El debugging es trivial ejecutando
el SQL directamente en cualquier cliente PostgreSQL. Comparado con las APIs específicas de
Qdrant, Milvus o Weaviate, pgvector ofrece máxima transparencia y claridad.

2. ¿Qué ventajas ofrece el mundo SQL (JOIN, filtros, agregaciones)?

SQL ofrece ventajas significativas: primero, JOINs nativos permiten combinar búsqueda
vectorial con datos relacionales sin necesidad de dos sistemas separados. Segundo, filtros
complejos usando WHERE con condiciones arbitrarias, subqueries y CTEs. Tercero, agregaciones
potentes con GROUP BY, AVG, MAX sobre scores de similitud. Cuarto, transacciones ACID que
garantizan consistencia. Quinto, un ecosistema maduro con herramientas de backup, replicación
y monitoreo. Finalmente, un solo sistema para datos vectoriales y estructurados, simplificando
la arquitectura y evitando sincronización entre múltiples bases de datos.

3. ¿Qué limitaciones esperas en escalabilidad frente a bases vectoriales dedicadas?

Las limitaciones son notables en varios aspectos. El rendimiento sufre con millones de
vectores: bases dedicadas como Milvus son 2-10x más rápidas. PostgreSQL no tiene sharding
nativo para vectores, requiriendo soluciones manuales o extensiones como Citus. Los índices
disponibles (HNSW, IVFFlat) son menos eficientes que implementaciones dedicadas y no permiten
la configuración avanzada que ofrecen sistemas especializados. Falta compresión de vectores
(PQ, SQ) y otras optimizaciones específicas para vectores. La gestión de memoria no está
optimizada para cargas vectoriales masivas. Sin embargo, pgvector es ideal para <1M vectores,
cuando necesitas JOINs complejos, ya tienes PostgreSQL en producción, o las transacciones
ACID son críticas.